In [1]:
import pandas as pd
import numpy as np
import SimpleITK
import os
import matplotlib.pyplot as plt

In [2]:
def load_dicom_series(input_dir):
    """Reads an entire DICOM series of slices from 'input_dir' and returns its pixel data as an array."""

    
    try:

        reader = SimpleITK.ImageSeriesReader()
        dicom_names = reader.GetGDCMSeriesFileNames(input_dir)
        reader.SetFileNames(dicom_names)
        dicom_series = reader.Execute()

        dicom_array = SimpleITK.GetArrayFromImage(dicom_series)
        dicom_array = np.moveaxis(dicom_array, 0, -1)
        return dicom_array
    except:
        return None

In [4]:
def load_ktrans(input_dir):
    try:
        files = os.listdir(input_dir)

        mhd_file = [f for f in files if ".mhd" in f]
        mhd_file = mhd_file[0]


        path = os.path.join(input_dir, mhd_file)
        itkimage = SimpleITK.ReadImage(path)
        ct_scan = SimpleITK.GetArrayFromImage(itkimage)
        
        ct_scan = np.moveaxis(ct_scan, 0, -1)

        return ct_scan
    except: 
        return None
    

In [5]:
def get_image_patch(img_array, coords, padding=32):


    """ Description
    :type img_array: numpy array
    :param img_array: a image comprised of a numpy array

    :type coords: list(i,j)
    :param coords: The coordinates where the crop will be centered, of type (i,j), i being the columns and j the row

    :type padding: int or list(x,y)
    :param padding: The padding that will be around the center coordinates. If an int, it will create a square image. If a list x is the horizontal padding and y the vertical

    :raises:

    :rtype:
     """

    i = coords[0]
    j = coords[1]

    if isinstance(padding, list):
        h_padding = padding[0]
        v_padding = padding[1]
    else:
        h_padding = padding
        v_padding = padding

    X_ = img_array[j - v_padding: j + v_padding, i - h_padding: i + h_padding]
    # NUMPY ARRAYs are of standart (row, columns)

    return X_

In [8]:
def get_exam(lesion_info, exam='ADC', padding=32, path=base_path):
    
    
    exame_row = lesion_info
    
    exame_row = exame_row.loc[exame_row.DCMSerDescr.str.contains(exam)]
    
    if exame_row.empty:
        print("No lesion found")
        return None
    
    else:
        tmp_row = exame_row.iloc[0]
        
        exam_folder = os.path.join(base_path, tmp_row.ProxID, tmp_row.DCMSerDescr)
    
        if(exam != 'KTrans'):
            image = load_dicom_series(input_dir=exam_folder)

        else:
            image = load_ktrans(exam_folder)
        
        if image is None:
            return None
        
        
        if(tmp_row.k < image.shape[2]):
            slice_array = image[:,:, tmp_row.k]
            patch = get_image_patch(slice_array, (tmp_row.i, tmp_row.j), padding=padding)
            # print(patch.shape)

        else:
            print("Had to cut image")
            return None
            
        if(patch.shape == (2*padding,2*padding)):
            return np.asarray(patch).reshape((1,2*padding,2*padding, 1))
        
        
        
        
    
    

In [7]:
metadata = pd.read_csv("../data/interim/train_information.csv")
base_path = "../data/interim/train/"

In [9]:
to_iterate = metadata.drop_duplicates(["ProxID", "fid", "pos"])[["ProxID", "fid", "ClinSig"]]

In [10]:
from skimage.transform import rescale, resize
from imgaug import augmenters as iaa


In [13]:
# ADC
# t2-tse
# t2-loc

padding = 32

X = np.empty((1000, 64,64,3))

y = []
i = 0

aug = iaa.Scale(0.5, interpolation="area")
for tup in to_iterate.itertuples():
    X_ = []
    
    person_path = os.path.join(base_path, tup.ProxID)
    
    person_exams = os.listdir(person_path)
    
    
    lesion_info = metadata[(metadata.ProxID == tup.ProxID) & (metadata.fid == tup.fid)]
    

    adc = get_exam(lesion_info, '_ADC', padding=padding )
    if adc is  None:
        continue
        

    t2_tse = get_exam(lesion_info, 't2_tse_tra', padding=2*padding)
    #print(t2_tse.shape)
    if t2_tse is None:
        continue
    #print(t2_tse[0].shape)
    t2_tse = np.uint8(t2_tse)
    t2_tse = aug.augment_images(t2_tse)
    
    ktrans = get_exam(lesion_info, 'KTrans', padding=padding)
    
    if ktrans is None:
        continue

    X_ = np.concatenate([adc, t2_tse, ktrans], axis = 3)
    # X = np.append(X, X_, axis=0)
    X[i, :, :, :] = X_
    i = i+1
    
    y_ = 1 if tup.ClinSig else 0
    y.append(y_)
    
    

X = X[:i]

print(X.shape)

Had to cut image
Had to cut image
(0, 64, 64, 3)


In [14]:
import matplotlib.pyplot as plt

plt.imshow(X[2, :, :, 1])

IndexError: index 2 is out of bounds for axis 0 with size 0

In [ ]:
plt.imshow(X[1, :, :, :])

In [ ]:
lesion_info = metadata.loc[(metadata.ProxID == tmp.ProxID) & (metadata.fid == tmp.fid)]
lesion_info
get_exam(tmp,lesion_info, 'ADC')

In [ ]:
X[0] = np.zeros((64,64,2))

In [ ]:
X.shape

In [ ]:
img = load_dicom_series("/home/paulo/Projects/thesis/prostatex/data/interim/train/ProstateX-0001/t2_tse_tra")

In [ ]:
img.shape

In [ ]:
img_ = img[:,:, 10]

In [ ]:
plt.imshow(img_)

In [ ]:
patch_ = get_image_patch(img_array=img_, coords=(130,200), padding=128)

In [ ]:
plt.imshow(patch_)

In [ ]:
np.save("../data/processed/X_2c.npy", X)
np.save("../data/processed/y_2c.npy", y)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[5,:,:,:], interpolation="nearest")

In [ ]:
mhd = load_ktrans('/home/paulo/Projects/thesis/prostatex/data/interim/train/ProstateX-0001/KTrans')
mhd.shape

In [ ]:
plt.imshow(mhd[10, : , : ])

In [ ]:
mhd_ = np.moveaxis(mhd, 0, -1)

mhd_.shape

In [ ]:
plt.imshow(mhd_[:, : , 10])

In [11]:
to_iterate.head()

,ProxID,fid,ClinSig
0,ProstateX-0000,1,True
9,ProstateX-0001,1,False
22,ProstateX-0002,2,False
33,ProstateX-0002,1,True
44,ProstateX-0003,1,False
